In [1]:
import cProfile
import random as rd
import time
from collections import defaultdict
import numpy as np
from itertools import cycle
import Utilities.experiment_utils as eu

import Games.mnk as mnk
import Agents.random as arand
import Agents.vanilla_mcts as mcts 
#import carcassonne as carc

In [2]:
base_gs = mnk.GameState(3,3,3)
base_gs.set_initial_state()
root_node = mcts.Node(state=base_gs)
mcts_agent = mcts.MCTS_Player(rollouts=50, c=1, max_iterations=-1, default_policy = arand.RandomPlayer())
_ = mcts_agent.choose_action(base_gs)

Vanilla_MCTS : Random move returned


In [3]:

mcts_agent.iteration(mcts_agent.root_node)
print(mcts_agent.view_mcts_tree())

In [2]:
base_gs = mnk.GameState(3,3,3)
base_gs.set_initial_state()
agent1 = arand.RandomPlayer()
agent2 = mcts.MCTS_Player(rollouts=10, c=1, max_iterations=20, default_policy=arand.RandomPlayer())
players = [agent1,agent2]
gs,logs = eu.play_game(base_gs, players, True)
gs.view_game_state()
print(logs)

[[' ' 'O' 'O']
 [' ' 'O' 'X']
 ['X' 'X' 'X']]
         player player_turn chosen_action turn had_n_options      time
0        Random           0    (x1,y2,t0)    1             9       0.0
1  Vanilla_MCTS           1    (x0,y1,t1)    2             8 -0.014945
2        Random           0    (x2,y2,t0)    3             7       0.0
3  Vanilla_MCTS           1    (x1,y1,t1)    4             6 -0.003984
4        Random           0    (x2,y1,t0)    5             5       0.0
5  Vanilla_MCTS           1    (x0,y2,t1)    6             4 -0.000957
6        Random           0    (x2,y0,t0)    7             3       0.0


In [8]:
view_mcts_tree(agent2.root_node, agent2)

edge_action:None, visits:22, avg_reward:-0.282, children:4 ucb:None
--edge_action:(x0,y0,t1), visits:9, avg_reward:0.0956, children:3 ucb:0.924
----edge_action:(x2,y1,t0), visits:2, avg_reward:0.56, children:1 ucb:0.922
------edge_action:(x2,y2,t1), visits:1, avg_reward:1, children:0 ucb:2.18
----edge_action:(x0,y1,t0), visits:2, avg_reward:0.745, children:1 ucb:0.737
------edge_action:(x2,y2,t1), visits:1, avg_reward:1, children:0 ucb:2.18
----edge_action:(x2,y2,t0), visits:4, avg_reward:-0.4, children:2 ucb:1.45
------edge_action:(x2,y1,t1), visits:2, avg_reward:0, children:1 ucb:1.18
--------edge_action:(x0,y1,t0), visits:1, avg_reward:0, children:0 ucb:1.18
------edge_action:(x0,y1,t1), visits:1, avg_reward:-1, children:0 ucb:0.665
--edge_action:(x0,y1,t1), visits:3, avg_reward:-0.847, children:2 ucb:0.589
----edge_action:(x0,y0,t0), visits:1, avg_reward:-1, children:0 ucb:2.48
----edge_action:(x2,y1,t0), visits:1, avg_reward:-1, children:0 ucb:2.48
--edge_action:(x2,y1,t1), visits

In [5]:
base_gs = mnk.GameState(3,3,3)
base_gs.set_initial_state()
start = time.time()
games = 0
agent = arand.RandomPlayer()
wins = defaultdict(lambda: 0)

def random_games(games, state):
    wins = defaultdict(lambda:0)
    #aan = []
    for _ in range(games):
        gs = state.duplicate()
        #aan.append(len(gs.available_actions))
        while not gs.is_terminal:
            action = agent.choose_action(gs)
            gs.make_action(action)
        wins[gs.winner] += 1
        gs.view_game_state()
    #print(aan)
    return wins

def random_game(state):
    gs = state.duplicate()
    while not gs.is_terminal:
        action = agent.choose_action(gs)
        gs.make_action(action)
    return gs


wins =  random_games(10, base_gs)
print(str(wins))

[['O' 'O' 'X']
 ['X' 'X' 'O']
 ['X' 'O' 'X']]


IndexError: list index out of range

In [4]:
cProfile.run("wins =  random_games(10000, base_gs)")

         10264365 function calls in 12.952 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.623    0.623   12.952   12.952 1794207304.py:8(random_games)
        2    0.000    0.000    0.000    0.000 1794207304.py:9(<lambda>)
        1    0.000    0.000   12.952   12.952 <string>:1(<module>)
   878085    0.971    0.000    4.956    0.000 mnk.py:105(_update_available_actions)
   878085    3.867    0.000    3.867    0.000 mnk.py:106(<listcomp>)
   888085    4.734    0.000    9.781    0.000 mnk.py:108(make_action)
    10000    0.064    0.000    0.643    0.000 mnk.py:157(duplicate)
    20000    0.332    0.000    0.332    0.000 mnk.py:161(<dictcomp>)
    10000    0.060    0.000    0.060    0.000 mnk.py:162(<listcomp>)
    10000    0.171    0.000    0.171    0.000 mnk.py:163(<dictcomp>)
    10000    0.011    0.000    0.011    0.000 mnk.py:37(__init__)
   888085    0.659    0.000    0.936    0.000 random.py:239(_randbe

In [5]:
wins =  random_games(1000, base_gs)
print(wins)

defaultdict(<function random_games.<locals>.<lambda> at 0x000001AF1C4F5820>, {0: 491, 1: 509})


In [4]:
base_gs = mnk.GameState(5,5,3)
base_gs.set_initial_state()
agent = ag.RandomPlayer()
gs = base_gs.duplicate()

In [8]:

action = agent.choose_action(gs)
gs.make_action(action)
horizontal = gs.board[action.x]
vertical = gs.board[:,action.y]
offset = action.y-action.x
antioffset = gs.n - 1 - action.y - action.x
diagonal_up = gs.board.diagonal(offset)
diagonal_down = np.fliplr(gs.board).diagonal(antioffset)


gs.view_game_state()
print("horizontal", horizontal)
print("vertical", vertical.transpose())
print("action.x", action.x)
print("action.y", action.y)
print("offset", offset)
print("diagonal_up", diagonal_up)
print("diagonal_down", diagonal_down)

[[' ' ' ' ' ' ' ' ' ']
 ['O' 'X' ' ' ' ' ' ']
 [' ' ' ' 'X' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' 'O' ' ' ' ' ' ']]
horizontal [[2 1 2 2 2]]
vertical [[2 0 2 2 1]]
action.x 4
action.y 1
offset -3
diagonal_up [[2 1]]
diagonal_down [[2 2 2 1]]
